In [1]:
#spirograph project from the Python Playground book -> I'm pretty much just looking at the book and doing what it says
#check log notes for math and details beyond comments 

In [23]:
import math
import turtle #the module used to create the drawings
import sys, random, argparse
import numpy as np
import random
from PIL import Image
from datetime import datetime

In [24]:
#function used to draw the circle
#3 parameters used in equations
#if there is an error, run again
'''def DrawCirc(x,y,r):
    #move to start of the circle
    turtle.up()#like moving the pen off of the paper
    turtle.setpos(x + r ,y)#set the position of the pen - (x,y) is the center of the circle
    turtle.down()#putting the pen back on the paper
    for i in range(0,365,5): #range 0-360 which increments of i in steps of 5
        #i = angle parameter theta
        a = math.radians(i) #convert angle from degrees to radians (what most computers require)
        turtle.setpos(x + r*math.cos(a), y + r*math.sin(a))
        #math in notes(1)
DrawCirc(100,100,50)
turtle.mainloop()'''

'def DrawCirc(x,y,r):\n    #move to start of the circle\n    turtle.up()#like moving the pen off of the paper\n    turtle.setpos(x + r ,y)#set the position of the pen - (x,y) is the center of the circle\n    turtle.down()#putting the pen back on the paper\n    for i in range(0,365,5): #range 0-360 which increments of i in steps of 5\n        #i = angle parameter theta\n        a = math.radians(i) #convert angle from degrees to radians (what most computers require)\n        turtle.setpos(x + r*math.cos(a), y + r*math.sin(a))\n        #math in notes(1)\nDrawCirc(100,100,50)\nturtle.mainloop()'

In [25]:
'''a class that draws a spirograph (a class is like a blueprint for creating object
-> describes states and behaviors)'''
#an object has states(color, name) and behaviors(eating, walking)

class Spiro:
    def __init__(self, xc, yc, col, R, r, l): #notes(1+2)
        self.t = turtle.Turtle() #creates the turtle object -> draw multiple spiros simultaneously
        self.t.shape("turtle") #set the cursor shape
        self.step = 5 #set the step in degrees/set angle increment
        self.drawingComplete = False #set the drawing complete flag -> produces a bunch of spiros
        self.setparams(xc, yc, col, R, r, l) #set parameters
        self.restart() #initialize the drawing


#set the parameters
    def setparams(self, xc, yc, col, R, r, l):
#store the coordinates of the center of the curve
        self.xc = xc
        self.yc = yc
#convert the radius of each circle to an integer and store the values
        self.R = int(R)
        self.r = int(r)
        self.l = l
        self.col = col
#reduce r/R (see notes) to its smallest form by dividing with the GDC
        gcdVal = gcd(self.r, self.R)#determine periodicity of the curve
        self.nRot = self.r//gcdVal #save the periodicity
#get ratio of the radii
        self.k = r/float(R)
#set the color
        self.t.color(*col)
#store the current angle -> used to create the animation
        self.a = 0

#restart the drawing
    def restart(self):
    #set the flag
        self.drawingComplete = False #determine whether or not the drawing has been completed (currently initializing)
    #show the turtle
        self.t.showturtle()
    #go up to the first point
        self.t.up()
        R, k, l = self.R, self.k, self.l #using local variables to keep the code compact
        a = 0.0
    #math in notes (parametric)
        x = R*((1 - k)*math.cos(a) + l*k*math.cos((1 - k)*a/k))
        y = R*((1 - k)*math.sin(a) - l*k*math.sin((1 - k)*a/k))
        self.t.setpos(self.xc + x, self.yc + y) #starting position of drawing
        self.t.down()

#draw the whole thing
    def draw(self):
    #draw the rest of the points
        R, k, l = self.R, self.k, self.l
        for i in range(0, 360*self.nRot + 1, self.step):
            a = math.radians(i)
            x = R*((1 - k)*math.cos(a) + l*k*math.cos((1 - k)*a/k))
            y = R*((1 - k)*math.sin(a) - l*k*math.sin((1 - k)*a/k))
            self.t.setpos(self.xc + x, self.yc + y)
    #drawing is now done, so hide the turtle cursor
        self.t.hideturtle()

    '''The update() method shows the drawing method you use to draw the curve
segment by segment to create an animation.'''
    #update by one step
    def update(self):
        if self.drawingComplete==True:
            return #this skips the rest of the steps if the drawing is done
    #incriments the angle
        self.a += self.step
    #draw a step
        R, k, l = self.R, self.k, self.l
    #set the angle
    #calcs the (x,y) position corresponding to the current angle and moves the cursor there
        a = math.radians(self.a)
        x = self.R*((1 - k)*math.cos(a) + l*k*math.cos((1 - k)*a/k))
        y = self.R*((1 - k)*math.sin(a) - l*k*math.sin((1 - k)*a/k))
        self.t.setpos(self.xc + x, self.yc + y)    
    #if drawing is complete, set the flag
        if self.a >= 360*self.nRot:
            self.drawingComplete = True
        #drawing is now done, hide the cursor
            self.t.hideturtle()
            
    #clear everything
    def clear(self):
        self.t.clear()

In [27]:
'''spiroAnimator class lets random spiros be drawn simultaneously
 -> uses a timer to draw curves one seg at a time'''
#a class for animating spirographs
class SpiroAnimator:
    #constructor
    def __init__(self, N):
        #set the timer value in milliseconds
        self.deltaT = 10
        #get the window dimensions
        self.width = turtle.window_width()
        self.height = turtle.window_height()
        #create the spiro objects/ empty array that will be populated with Spiro objects
        #-> N is how many the drawing will loop
        self.spiros = []
        for i in range(N):
            #generate random parameters
            rparams = self.genRandomParams()
            #set the spiro parameters
            #* converts rparams into a list of arguments(is a tuple)
            # R, k, l = self.R, self.k, self.l is a tuple (like a list)
            spiro = Spiro(*rparams)
            self.spiros.append(spiro)
            #call timer
            turtle.ontimer(self.update, self.deltaT)
        

#generate random parameters -> creates random curves
    def genRandomParams(self):
        width, height = self.width, self.height
        '''set R to a random integer between 50 and the value of half 
    the smallest dimension of your window'''
        '''set r to between 10 and 90 percent of R'''
        '''set l to a random fraction between 0.1 and 0.9'''
        R = random.randint(50, min(width, height)//2)
        r = random.randint(10, 9*R//10)
    #uniform() is randint() for floats
        l = random.uniform(0.1, 0.9)
        '''select a random point on the screen to place the center of the
spiro by selecting random x- and y-coordinates from within the screen
boundaries'''
        xc = random.randint(-width//2, width//2)
        yc = random.randint(-height//2, height//2)
    #assign a random color
        col = (random.random(),random.random(),random.random())
    #calculated params are returned as a tuple
        return(xc, yc, col, R, r, l)

#restarting the spiro drawing
    def restart(self):
        for spiro in self.spiros:
        #clear
            spiro.clear()
        #generate random parameters
            rparams = self.genRandomParams()
        #set the spiro parameters
            spiro.setparams(*rparams)
        #restart drawing
            spriro.restart()    

    def update(self):
        #update all spiros
        nComplete = 0 #counter
        for spiro in self.spiros:
        #update
            spiro.update()
        #count complted spiros
            if spiro.drawingComplete:
                nComplete += 1
    #restart if all spiros are complete
        if nComplete == len(self.spiros):
            self.restart()
    #call the timer
        turtle.ontimer(self.update, self.deltaT)
    
    #toggle cursor on and off
    def toggleTurtles(self):
        for spiro in self.spiros:
            if spiro.t.isvisible():
                spiro.t.hideturtle()
            else:
                spiro.t.showturtle()
                        

#save the drawings as png image files
def saveDrawing():
    #hide the cursor
    turtle.hideturtle()
    #generate unique filenames
    dateStr =(datetime.now()).strftime("%d%b%Y-%H%M%S")
    fileName = "spiro" + dateStr
    print("saving drawing to %s.eps/png" % fileName)
    #get the tkinter canvas -> creates user interface
    canvas = turtle.getcanvas()
    #save the drawing as a postscript image -> high resolution
    canvas.postscript(file =fileName + ".eps")
    # use the Pillow module to convert the postscript image file to PNG -> more versitile
    img = Image.open(fileName + ".eps")
    img.save(fileName + ".png", "png")
    #show the cursor
    turtle.showturtle()

#using the main() method to parse command line options sent to the program
#create the arg parser obj
def main():
     # use sys.argv if needed
    print('generating spirograph...')
     # create parser
    descStr = """This program draws Spirographs using the Turtle module.
 When run with no arguments, this program draws random Spirographs."""

    parser = argparse.ArgumentParser(description=descStr)

    #add expected args
    parser.add_argument('--sparams', nargs=3, dest='sparams', required=False,
 help="The three arguments in sparams: R, r, l.")

    #parse args/ call that does the actual parsing
    #args = parser.parse_args()
    args, unknown = parser.parse_known_args()

    #set width of the drawing window to 80% of teh screen width
    turtle.setup(width = 0.8)
    #set the cursor shape to turtle
    turtle.shape('turtle')
    #set the title to Spirographs!
    turtle.title("Spirographs!")
    #add key handler to save drawings -> save drawing when you press the s key
    turtle.onkey(saveDrawing, "s")
    #start listening -> listen for user events/input
    turtle.listen()
    #hide the main turtle cursor
    turtle.hideturtle()
    
    #check for any args sent to --sparams and draw the spirograph
    if args.sparams:
        # you first check whether any arguments were given to --sparams; convert them into floats if so
        params = [float(x) for x in args.sparams]
        #draw the spirograpgh with the given parameters
        col = (0.0, 0.0, 0.0)
        spiro = Spiro(0, 0, col, *params)
        spiro.draw()
    else:
        #create an animator object
        #creates number of drawing specified
        spiroAnim = SpiroAnimator(4)
        #add a key handler to toggle the turtle cursor -> the t key
        turtle.onkey(spiroAnim.toggleTurtles, "t")
        #add a key handler to restart the animation -> spacebar
        turtle.onkey(spiroAnim.restart, "space")
        
    #start the turtle main loop -> tells tkinter window to stay open
    turtle.mainloop()
    
#call main
if __name__ == "__main__":
    main()
    

generating spirograph...


/Users/student/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/ipykernel/__main__.py:26: DeprecationWarning: fractions.gcd() is deprecated. Use math.gcd() instead.


saving drawing to spiro02Aug2019-131625.eps/png


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/student/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/tkinter/__init__.py", line 1550, in __call__
    return self.func(*args)
  File "/Users/student/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/turtle.py", line 686, in eventfun
    fun()
  File "<ipython-input-27-cabf04d9710d>", line 98, in saveDrawing
    img.save(fileName + ".png", "png")
  File "/Users/student/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/PIL/Image.py", line 1698, in save
    self.load()
  File "/Users/student/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/PIL/EpsImagePlugin.py", line 338, in load
    self.im = Ghostscript(self.tile, self.size, self.fp, scale)
  File "/Users/student/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/PIL/EpsImagePlugin.py", line 143, in Ghostscript
    stdout=subprocess.PIPE)
  File "/Users/student/Library/Enthought/Canopy/edm/envs/U

saving drawing to spiro02Aug2019-131718.eps/png


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/student/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/tkinter/__init__.py", line 1550, in __call__
    return self.func(*args)
  File "/Users/student/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/turtle.py", line 686, in eventfun
    fun()
  File "<ipython-input-27-cabf04d9710d>", line 98, in saveDrawing
    img.save(fileName + ".png", "png")
  File "/Users/student/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/PIL/Image.py", line 1698, in save
    self.load()
  File "/Users/student/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/PIL/EpsImagePlugin.py", line 338, in load
    self.im = Ghostscript(self.tile, self.size, self.fp, scale)
  File "/Users/student/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/PIL/EpsImagePlugin.py", line 143, in Ghostscript
    stdout=subprocess.PIPE)
  File "/Users/student/Library/Enthought/Canopy/edm/envs/U

TclError: invalid command name ".4974258944"